In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [69]:
url_inicial = 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=CategoryIndex&CategoryPager_page=7&sort_field=posicion&sort_order=DESC&category=5039'
url_root = 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=CategoryIndex&CategoryPager_page=7&sort_field=posicion&sort_order=DESC&category=5039'
r=requests.get(url_inicial)

In [70]:
soup = BeautifulSoup(r.text, 'html.parser')

In [71]:
lista_articulo = soup.find_all('div', class_='cat_prod')
links_libros = [x.find('a').get('href') for x in lista_articulo]
links_libros = [urljoin (url_root, i) for i in links_libros]
links_libros

['https://riiit.com.mx/item/httpriiitcommxappssitefilesthe_led_erik_immx_2_1pdf.html?id=5039&id=5039',
 'https://riiit.com.mx/item/httpriiitcommxappssitefilesrtesis_itesco_ver_2pdf.html?id=5039&id=5039',
 'https://riiit.com.mx/item/httpriiitcommxappssitefilesportuitssna_20_3pdf.html?id=5039&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84952&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84953&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84954&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84955&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84703&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=ViewItem&id=5039&item_id=84704&id=5039',
 'https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=View

In [72]:
def get_url_items(soup,url):
    lista_articulo = soup.find_all('div', class_='cat_prod')
    links_libros = [x.find('a').get('href') for x in lista_articulo]
    links_libros = [urljoin (url_root, i) for i in links_libros]
    return links_libros

In [73]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://riiit.com.mx/apps/site/idem.php?module=Catalog&action=CategoryIndex&CategoryPager_page=7&sort_field=posicion&sort_order=DESC&category=5039


In [74]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

15

In [75]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [76]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Internacional de Investigación e Innovación Tecnológica'
    a='Multidisciplinaria'
    tem='Innovación Tecnológica'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('p', class_='title_item').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='description_item_art').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='ficha').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [77]:
list_scraper=list_scraper[0:15]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14


In [78]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,LED as a low cost single photon source. Articu...,Las fuentes de fotones individuales usan luz n...,None,https://riiit.com.mx/item/httpriiitcommxappssi...,http://riiit.com.mx/apps/site/files/the_led._e...
1,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Diseño y simulación de Órtesis de descarga par...,Legg  Calvé - Perthes es una enfermedad juven...,None,https://riiit.com.mx/item/httpriiitcommxappssi...,http://riiit.com.mx/apps/site/files/rtesis2c_i...
2,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Plan de acción para mejorar la función tutoria...,El presente trabajo propone el portal web PORT...,None,https://riiit.com.mx/item/httpriiitcommxappssi...,http://riiit.com.mx/apps/site/files/portuitssn...
3,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Estudio teórico de las propiedades estructural...,El TiO2 es un semiconductor activo utilizado e...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/tania_d._t...
4,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Synthesis and characterisation of electrospun ...,Nanofibras de óxido de polietileno enriquecida...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/penelope_r...
5,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Modelo para la ejecución de pruebas de software,Las pruebas de software son un elemento crític...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/paul_j._c....
6,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Optimización del Diseño de Protecciones Solare...,Este estudio analiza la influencia de diferent...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/abigail_im...
7,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Análisis de la Temperatura en el Comportamient...,La pérdida de potencia de un sistema fotovolta...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/bimestre_j...
8,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,Sistema opto-mecánico θ-2θ para su uso en Reso...,En este trabajo se muestra el diseño e impleme...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/bimestre_j...
9,Revista Internacional de Investigación e Innov...,Multidisciplinaria,Innovación Tecnológica,"PV module defects detection, a study using ele...",Algunos defectos pueden surgir posterior a la ...,None,https://riiit.com.mx/apps/site/idem.php?module...,http://riiit.com.mx/apps/site/files/david_mtz....


In [79]:
df_catalogo.to_csv('Revista211.07.csv', index=False)